In [ ]:
# ===== CONSTANTS =====
# cache_file: Dateiname für Zwischenergebnisse
# source_file: Ursprungsdatei mit vollständigen Namensdaten
cache_file = "cache.csv"
source_file = "names.csv"

# ===== CORE FUNCTIONS =====

# Lies die CSV-Datei ein und gib Header + alle Zeilen zurück
# file: Pfad zur CSV-Datei
# Rückgabe: header (Liste von Spaltennamen), lines (Liste aller Zeilen als Strings)
def get_content(file):
    # Öffne die Datei im Lesemodus ("r" = read)
    # Streng nach Python-Fluss: Fehler wirft Exception, wenn Datei fehlt
    with open(file, "r") as data:
        lines = data.readlines()                  # Lese alle Zeilen
        header = lines[0].strip().split(",")    # Erste Zeile = Header, entferne Zeilenumbruch
    return header, lines

# Erstelle oder überschreibe die Cache-Datei mit den übergebenen Zeilen
def generate_cache_file(header, lines, file=cache_file):
    with open(file, "w") as new_file:  # "w" = write, überschreibt bestehende Datei
        # Schreibe Header-Zeile, füge Zeilenumbruch hinzu
        new_file.write(",".join(header) + "
")
        # Jede Zeile hier ist bereits Liste von Spaltenwerten
        for line in lines:
            new_file.write(",".join(line) + "
")

# Zeige die Cache-Datei als formatiertes ASCII-Table in Konsole
def print_formatted_table(file=cache_file):
    header, lines = get_content(file)
    # Infozeilen
    print("\nFormatted Table, from file:", file)
    print("With a total of " + str(len(lines)-1) + " entrys.")
    # Trennlinie basierend auf Spaltenzahl
    print("-" * round(23.2 * len(header)))

    # Headerzeile: jede Spalte 20 Zeichen breit
    print(("| {:<20} " * len(header) + "|").format(*header))
    print("-" * round(23.2 * len(header)))

    # Datenzeilen: splitte an Komma und formatiere gleich
    for line in lines[1:]:
        cells = line.strip().split(",")  # Teile Zeile in Zellen
        print(("| {:<20} " * len(header) + "|").format(*cells))

# Filtere Zeilen nach Spaltenwerten
def get_rows_by_value(column, value, file=cache_file, return_table=False):
    header, lines = get_content(file)
    # Prüfe gültige Spalte
    if column not in header:
        print(f"Spalte '{column}' nicht gefunden.")
        return
    col_index = header.index(column)  # Finde Spaltenindex
    rows = []  # Neue Liste für gefilterte Zeilen
    for line in lines[1:]:
        cells = line.strip().split(",")
        # Wenn Zellenwert in erlaubten Werten liegt, füge zur Ergebnisliste
        if cells[col_index] in value:
            rows.append(cells)

    # Speichere resultierende Zeilen in cache
    generate_cache_file(header, rows)
    # Ausgabe der Filter-Info
    print("\nOnly returned rows with value in column:", column)
    print("For Values:", value)
    if return_table:
        print_formatted_table()

# Sortiere mit einfachem Bubble-Sort nach einer Spalte
def sort_by(column, area, reverse=False, file=cache_file, return_table=False):
    header, lines = get_content(file)
    if column not in header:
        print(f"Spalte '{column}' nicht gefunden.")
        return
    # Bereich definieren: [Start, Ende]
    if area == []:
        area = [1, len(lines)]
    # Erstelle Liste von Zeilen (als Listen)
    rows = [line.strip().split(",") for line in lines[area[0]:area[1]]]
    idx = header.index(column)
    # Bubble-Sort: vergleiche benachbarte Paare
    for i in range(len(rows)):
        for j in range(len(rows) - i - 1):
            a = int(rows[j][idx])
            b = int(rows[j+1][idx])
            # Prüfe Reihenfolge und tausche ggf.
            if (a > b and not reverse) or (a < b and reverse):
                rows[j], rows[j+1] = rows[j+1], rows[j]

    generate_cache_file(header, rows)
    print("\nSorted by column:", column)
    print("Range:", area[0], "to", area[1])
    if return_table:
        print_formatted_table()

# Summiere alle Häufigkeitswerte in Spalte "Number"
def count_humans(file=cache_file):
    header, lines = get_content(file)
    idx = header.index("Number")
    total = 0
    for line in lines[1:]:
        cells = line.strip().split(",")
        total += int(cells[idx])
    return total

# Berechne Durchschnittsalter: CURRENT_YEAR minus Geburtsjahr

def average_age(file=cache_file):
    header, lines = get_content(file)
    idx = header.index("YearOfBirth")
    current_year = 2025  # Fest definiert
    total_age = 0
    count = 0
    for line in lines[1:]:
        cells = line.strip().split(",")
        total_age += current_year - int(cells[idx])
        count += 1
    if count == 0:
        print("No data available to calculate average age.")
        return None
    return total_age / count

# ===== MAIN FUNCTION =====
def main():
    # Endlosschleife für Benutzerinteraktion
    while True:
        # Frage: Filter ausführen?
        choice = input("\nWould you like to filter the data? (yes/no/quit): ").lower()
        if choice == 'quit':  # Beende Programm
            print("Exiting program...")
            break
        if choice == 'yes':
            # Schritt 1: Filter-Parameter abfragen
            print("\nAvailable columns: StateCode, YearOfBirth, Sex, Number")
            col = input("Enter column: ")
            vals = input("Enter comma-separated values: ").split(',')
            vals = [v.strip() for v in vals]
            # Wende Filter an
            get_rows_by_value(col, vals, source_file)
            print("\nFilter applied!")
            # Zeige gefilterte Daten optional
            if input("See filtered data? (yes/no): ").lower() == 'yes':
                print_formatted_table()
            # Statistiken anzeigen?
            if input("Show stats? (yes/no): ").lower() == 'yes':
                print(f"Total: {count_humans()}")
                avg = average_age()
                if avg is not None:
                    print(f"Avg age: {avg:.1f}")
            # Sortieroption anbieten?
            if input("Sort data? (yes/no): ").lower() == 'yes':
                sc = input("Column to sort by: ")
                start = int(input("Start row (1): ") or 1)
                end = int(input("End row (-1=all): ") or -1)
                dir = input("Direction (asc/desc): ")
                sort_by(sc, [start, end], dir=='desc')
        elif choice == 'no':
            # Zeige Original-Statistiken
            print(f"Total humans: {count_humans(source_file)}")
            avg = average_age(source_file)
            if avg is not None:
                print(f"Average age: {avg:.1f}")
            continue
        else:
            print("Bitte 'yes', 'no' oder 'quit' eingeben.")
            continue

# Starte Programm, wenn als Script ausgeführt
if __name__ == "__main__":
    main()